## Continu Inzicht Demo (Create DB tables)
> https://tscontinuinzicht.hkvservices.nl/ci-demo

**Imports**

In [ ]:
# import algemeen
from pathlib import Path

# imports base modules
from toolbox_continu_inzicht import Config, DataAdapter

**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

<pre style="font-size: 10pt">
GlobalVariables:
    rootdir: "data"
    calc_time: "2024-11-19"
    moments: [-24,0,24,48]

    LoadsWaterinfo:
            parameters: ["waterhoogte"]
            MISSING_VALUE: 999

    LoadsMatroos:
        website: "noos"
        model: "fews_rmm_km"
        parameters: ["WATHTE"]

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: ""
        parameters: [ "WATHTE [m][NAP][OW]" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsFewsLimburg:
        host: "http://109.232.44.231"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "App_Maas"
        parameters: [ "WATHTE" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: -999

DataAdapter:

    calculation_start_config:
        type: ci_postgresql_to_calculation_start
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    calculation_end_config:
        type: ci_postgresql_to_calculation_end
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    in_measuringstation_waterinfo:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "waterinfo"

    in_measuringstation_matroos:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "matroos"

    in_measuringstation_fews_hhnk:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "fews_hhnk"

    in_measuringstation_fews_limburg:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "fews_limburg"

    out_waterstanden_ci_postgresql_001:
        type: ci_postgresql_measuringstation_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 0.01

    out_waterstanden_ci_postgresql_1:
        type: ci_postgresql_measuringstation_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 1

    in_measuringstation_data_table:
        type: ci_postgresql_measuringstation_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    in_measuringstation_conditions_table:
        type: ci_postgresql_from_conditions
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    out_measuringstation_states_table:
        type: ci_postgresql_to_states
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"

    in_section_table:
        type: ci_postgresql_from_sections
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    in_sectionfractions_table:
        type: ci_postgresql_from_sectionfractions
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    out_waterstanden_section_ci_postgresql:
        type: ci_postgresql_section_load_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"        

    df_moment_waterstanden:
        type: python

    in_fragilitycurves_table:
        type: ci_postgresql_fragilitycurves_table
        database: geoserver
        schema: continuinzicht_demo_realtime
        measureid: 0

    in_measure_fragilitycurves_table:
        type: ci_postgresql_measure_fragilitycurves_table
        database: geoserver
        schema: continuinzicht_demo_realtime                

    in_section_load_from_data_table:
        type: ci_postgresql_section_load_from_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    out_section_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 5

    out_section_technical_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 100

    out_section_measure_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 101        

    in_section_conditions:
        type: ci_postgresql_section_thresholds_from_conditions_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    in_section_data_failure_probability:
        type: ci_postgresql_section_failure_probability_from_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    out_section_states:
        type: ci_postgresql_section_to_states
        database: geoserver
        schema: continuinzicht_demo_realtime
</pre>
</details>

In [ ]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-demo-create-db.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

**Aanmaken adapter**

In [ ]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Waterkeringsystemen inlezen uit shape-bestand en wegschrijven naar de database**

In [ ]:
# lees de waterkeringsystemen uit een shape-bestand via de data_adapter 'in_dikesystems_geojson'
df_waterkeringsystemen = data_adapter.input(input="in_dikesystems_geojson")

# sla de waterkeringsystemen op in de CI database (tabel 'dikesystems') via de data_adapter 'out_dikesystems_table'
data_adapter.output(output="out_dikesystems_table", df=df_waterkeringsystemen)

**Segmenten inlezen uit shape-bestand en wegschrijven naar de database**

In [ ]:
# lees de segmenten uit een shape-bestand via de data_adapter 'in_segments_geojson'
df_segmenten = data_adapter.input(input="in_segments_geojson")

# sla de segmenten op in de CI database (tabel 'segments') via de data_adapter 'out_segments_table'
data_adapter.output(output="out_segments_table", df=df_segmenten)

**Secties inlezen uit shape-bestand en wegschrijven naar de database**

In [ ]:
# lees de secties uit een shape-bestand via de data_adapter 'in_sections_geojson'
df_secties = data_adapter.input(input="in_sections_geojson")

# sla de secties op in de CI database (tabel 'sections') via de data_adapter 'out_sections_table'
data_adapter.output(output="out_sections_table", df=df_secties)

**Meetlocaties inlezen uit CSV en wegschrijven naar de database**

**What-if scenario's inlezen uit CSV en wegschrijven naar de database**

**What-if waterstanden inlezen uit CSV en wegschrijven naar de database**